# <span style="color: #FF0000">Exemple de machine de turing</span>

In [1]:
from ipywidgets import Button, Textarea, Layout,Label,HBox,HTML,Dropdown
from IPython.display import display,clear_output
import time
import threading

### Somme de deux binaires
La machine de Turing comporte ici 8 états
* e1 detecte le debut du binaire de gauche. passe à l'état e2 lorsque c'est le cas
* e2 detecte la fin du binaire de gauche. laisse les données inchangées et passe à l'état e3 lorsque c'est le cas
* e3 detecte la fin du binaire de droite, laisse les données inchangées et passe à l'état e4 lorsque c'est le cas
* e4 puis e5 balayent le binaire de droite pour lui retrancher 1. passe à l'état e6 si le binaire est non nul passe à l'état e8 si le binaire est nul
* e6 detecte la fin du binaire de gauche, laisse les données inchangées et passe à l'état e7 lorsque c'est le cas
* e7 ajoute 1 au binaire de gauche et passe à l'état e2 lorsque c'est le cas
* e8 efface le binaire de droite lorsqu'il est nul pour ne laisser sur le ruban que le resultat de l'addition. Met fin au calcul lorsque c'est fait.
  
Ainsi la machine de turing va détecter le binaire de droite, lui retrancher 1 puis détecter le binaire de gauche et lui ajouter 1 et ainsi de suite jusqu'à ce que le binaire de droite soit nul. Enfin, le binaire de droite est effacé pour ne laisser que le resultat sur le ruban.

In [2]:
Etat = {}
e1= {} #Detecte le debut du premier nombre et passe à e2
e1["0"] = ("e1","0","0","droite","e2")
e1["1"] = ("e1","1","1","droite","e2")
e1[" "] = ("e1"," "," ","droite","e1")
e2 = {} #Detecte le debut du deuxième nombre et passe à e3
e2["0"] = ("e2","0","0","droite","e2")
e2["1"] = ("e2","1","1","droite","e2")
e2[" "] = ("e2"," "," ","droite","e3")
e3 = {} #Detecte la fin du deuxième nombre et passe à e4
e3["0"] = ("e3","0","0","droite","e3")
e3["1"] = ("e3","1","1","droite","e3")
e3[" "] = ("e3"," "," ","gauche","e4")
e4 = {} # e4 + e5 Retranche un au nombre de droite 
e4["0"] = ("e4","0","0","gauche","e4")
e4["1"] = ("e4","1","0","droite","e5")
e4[" "] = ("e4"," "," ","droite","e8")
e5 = {}
e5["0"] = ("e5","0","1","droite","e5")
e5[" "] = ("e5"," "," ","gauche","e6")
e6 = {} # Detecte la fin du premier nombre
e6["0"] = ("e6","0","0","gauche","e6")
e6["1"] = ("e6","1","1","gauche","e6")
e6[" "] = ("e6"," "," ","gauche","e7")
e7 = {}
e7["0"] = ("e7","0","1","droite","e2")
e7["1"] = ("e7","1","0","gauche","e7")
e7[" "] = ("e7"," ","1","droite","e2")
e8={}
e8["0"] = ("e8","0"," ","droite","e8")
e8[" "] = ("e8"," "," ","gauche","fin")
Etat["fin"]=None
for i in range(1,9):
    Etat["e{}".format(i)]=eval("e{}".format(i))
liste = []
for k, v in Etat.items():
    if k!="fin":
        for k2, v2 in v.items():
            b = "|".join(v2)
            b=b.replace(" ","VIDE")
            liste.append("|"+b+"|")
a = "|Etat|Lit|Ecrit|Déplace|Suivant|\n|---|---|---|---|---|\n"+"\n".join(liste[:len(liste)//2])
b = "|Etat|Lit|Ecrit|Déplace|Suivant|\n|---|---|---|---|---|\n"+"\n".join(liste[len(liste)//2:])

<div class="row">
  <div class="col-xs-3 col-sm-3 col-lg-3 col-md-3" markdown="1">
  {{print(a)}}
  </div>
  <div class="col-xs-3 col-sm-3 col-lg-3 col-md-3" markdown="1">
  
  </div>
  <div class="col-xs-3 col-sm-3 col-lg-3 col-md-3" markdown="1">
  {{print(b)}}

</div>
  <div class="col-xs-3 col-sm-3 col-lg-3 col-md-3" markdown="1">
  
  </div>

Entrez 2 nombres séparés d'un case vide. Celui de droite sera diminué de 1 puis celui de gauche sera augmenté de 1 et ainsi de suite jusqu'à ce que celui de droite arrive à zéro. Le nombre de droite et ensuite effacé du ruban pour ne laisser que le résultat.

In [3]:
def on_value_change(change=None,i=None):
    global Ruban,dd,Rubani
    Rubani[i]=dd[i].value
    
def arret(b):
    global auto,stop,raz,lock
    stop.disabled=True
    raz.disabled=True
    auto=False
    while lock:
        pass
    stop.disabled=False
    raz.disabled=False

def debut(b):
    global go,Rubani,eiw,dd,step,e,c,auto,lock,stop,raz,premier_pas
    stop.disabled=True
    raz.disabled=True
    auto=False
    premier_pas = True
    while lock:
        pass
    stop.disabled=False
    raz.disabled=False
    Rubani=[" " for i in range(10)]
    Rubani[1]="1"
    Rubani[2]="0"
    Rubani[3]="1"
    Rubani[5]="1"
    Rubani[6]="1"
    for i in range(10):
        dd[i].value=Rubani[i]    
    clear_output()
    display(eiw)
    display(HBox([go,step,stop,raz]))
    for el in dd:
        el.disabled=False
    dd[0].disabled=True
    dd[-1].disabled=True
    e="e1"
    c=0
    
def on_button_click(b):
    global auto, premier_pas
    auto=True
    if premier_pas:
        suite()
        premier_pas = False
    else:
        suite2()
    
def work():
    global go,Ruban,Rubani,eiw,dd,step,e,c,auto,stop,raz, premier_pas
    turing()
    while auto and e!="fin":
        turing()
    go.disabled=False
    step.disabled=False
    if e =="fin":
        for el in dd:
            el.disabled=False
        dd[0].disabled=True
        dd[-1].disabled=True
        e = "e1"
        c = 0
        premier_pas = True
    return
    

def unpas(b):
    global auto, premier_pas
    auto = False
    if premier_pas:
        suite()
        premier_pas = False
    else:
        suite2()

def suite():
    global go,Ruban,Rubani,eiw,dd,step,e,c,auto,stop,raz
    clear_output()
    #display(titre)
    display(eiw)
    display(HBox([go,step,stop,raz]))
    l="".join(Rubani[:])
    l = l.strip().split(" ")
    for i in range(len(l)-1,-1,-1):
        if l[i]=="":
            l.pop(i)
    if len(l)!=2:
        return
    l=" ".join(l)
    l = l+" "
    while len(l)<10:
        l= " "+l
        
    for el in dd:
        el.disabled=True
    go.disabled=True
    step.disabled=True
    Ruban = list(l)
    affichage()
    thread = threading.Thread(target=work)
    thread.start()
    
def suite2():
    global go,Ruban,Rubani,eiw,dd,step,e,c,auto,stop,raz
    for el in dd:
        el.disabled=True
    go.disabled=True
    step.disabled=True
    thread = threading.Thread(target=work)
    thread.start()
    
def etatinit():
    global Ruban,dd,go,Rubani,titre,eiw,step,e,c,lock,stop,raz, premier_pas
    premier_pas = True
    lock=False
    Rubani=[" " for i in range(10)]
    Rubani[1]="1"
    Rubani[2]="0"
    Rubani[3]="1"
    Rubani[5]="1"
    Rubani[6]="1"
    e="e1"
    c=0
    dd = []
    for i in range(10):
        dd.append(Dropdown(
    options=[' ', '0', '1'],
    value=Rubani[i],
    description='',
    disabled=False,
    layout=Layout(width='48px',height='48px'),
))
    dd[0].disabled=True
    dd[-1].disabled=True
    
    titre=HTML(value = f"<center><b><font color='black'>Ruban Initial</b></center>", layout = Layout(width='100px',height='30px'))
    
    for i in range(10):
        dd[i].observe(lambda change,x=i:on_value_change(change,x),names="value")
    go = Button(description="GO",disabled=False,layout=Layout(width="100px",height="30px"))
    step = Button(description = "1 STEP",disabled=False,layout=Layout(width="100px",height="30px"))
    stop = Button(description = "STOP",disabled=False,layout=Layout(width="100px",height="30px"))
    raz = Button(description = "RESET",disabled=False,layout=Layout(width="100px",height="30px"))
    eiw=HBox([titre]+dd)
    #display(titre)
    display(eiw)
    display(HBox([go,step,stop,raz]))
    go.on_click(on_button_click)
    step.on_click(unpas)
    stop.on_click(arret)
    raz.on_click(debut)
    
def affichage():
    global Etat,Ruban,wcases,wetat,etatlab,entete,wentete,wetat,wcases,wruban
    titre2=HTML(value = f"<center><b><font color='black'>Ruban</b></center>", layout = Layout(width='100px',height='30px'))
    #display(titre2)
    
    entete = []
    enteteval =[" ","lit","ecrit","deplace","suivant"]
    for i in range(5):
        entete.append(HTML(value = f"<center><b><font color='black'>"+enteteval[i]+"</b></center>", layout = Layout(width='100px',height='30px')))
    wentete = HBox(entete)
    display(wentete)                  
    etatval = ["e1","1","1","droite","e1"]
    etatlab = []
    for i in range(5):
        etatlab.append(HTML(value = f"<center><b><font color='black'>"+etatval[i]+"</b></center>", layout = Layout(width='100px',height='30px')))
    wetat = HBox(etatlab)
    display(wetat)
    wcases = []
    for i in range(len(Ruban)):
        wcases.append(HTML(value = f"<center><b><font color='black'>"+Ruban[i]+"</b></center>", layout = Layout(width='48px',height='48px',border='3px solid black',)))
    wruban = HBox([titre2]+wcases)
    display(wruban)

def turing():
    global Etat,Ruban,wcases,wetat,etatlab,lock,e,c
    #etat = Etat[e][Ruban[case]]
    etatval = Etat[e][Ruban[c]]
    for i in range(5):
        etatlab[i].value = f"<center><b><font color='black'>"+etatval[i]+"</b></center>"
    for i in range(len(wcases)):
        wcases[i].layout.border = '3px solid black'

    wcases[c].layout.border = '3px solid red'
    
    for i in range(1,5):
        etatlab[i-1].layout.border = '0px'
        etatlab[i].layout.border = '3px solid red'
        time.sleep(0.25)
        if i==2:
            Ruban[c]=etatval[2]
            wcases[c].value = f"<center><b><font color='black'>"+Ruban[c]+"</b></center>"
        elif i == 3:
            wcases[c].layout.border = '3px solid black'
            if etatval[3]=="droite":
                c+=1
            else:
                c-=1
            wcases[c].layout.border = '3px solid red'
        elif i==4:
            e = etatval[4]
        time.sleep(0.5)
    etatlab[-1].layout.border = '0px'
    if e == "fin":
        wcases[c].layout.border = '3px solid black'
    lock = False
    return

etatinit()